In [1]:
import torch 
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
transform_train= transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
])
transform_test = transforms.Compose([
    transforms.ToTensor()
])

In [4]:
import torchvision 
trainset = torchvision.datasets.CIFAR100(root='/.data', train=True, download=True, transform=transform_train)
trainloader  = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

100%|██████████| 169M/169M [00:04<00:00, 35.0MB/s] 


In [ ]:
testset = torchvision.datasets.CIFAR100(root='/.data', train=False, download=True, transform=transform_test)
testloader  = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [6]:
len(trainset), len(testset)

(50000, 10000)

In [7]:
class_names  = trainset.classes
class_names[:10]

['apple',
 'aquarium_fish',
 'baby',
 'bear',
 'beaver',
 'bed',
 'bee',
 'beetle',
 'bicycle',
 'bottle']

In [28]:
model = torchvision.models.resnet18(weights=False)
model.fc = nn.Linear(model.fc.in_features, 100)
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [29]:
import torch.optim as optim
loss_fn  = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2, eta_min=0.01)

In [19]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_pred, y_true).sum().item()
    acc = correct/len(y_pred)
    return acc

In [31]:
#create a training loop
def train_step(model:torch.nn.Module,
               data_loader:torch.utils.data.dataloader,
               optimizer:torch.optim.Optimizer,
               loss_fn:torch.nn.Module,
               scheduler:torch.optim.lr_scheduler,
               accuracy_fn,
               device: torch.device):
    train_acc, train_loss = 0, 0 
    model.train()
    for batch, (X, Y) in enumerate(data_loader):
        X, Y = X.to(device), Y.to(device)
        y_pred  = model(X)
        loss = loss_fn(y_pred, Y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=Y, y_pred=y_pred.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"train loss: {train_loss:.5f}  | train acc: {train_acc:.5f}% | lr: {scheduler.get_last_lr()[0]:.5f}")

In [25]:
#create a testing loop
def test_step(model:torch.nn.Module, 
              data_loader:torch.utils.data.dataloader,
              loss_fn:torch.nn.Module,
              accuracy_fn,
              device:torch.device):
    test_acc, test_loss = 0, 0
    model.eval()
    with torch.inference_mode():
        for batch, (X,Y) in enumerate(data_loader):
            X, Y = X.to(device), Y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred , Y)
            test_loss += loss
            test_acc += accuracy_fn(y_true=Y, y_pred=y_pred.argmax(dim=1))
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"test loss: {test_loss:.5f} | test acc:{test_acc:.5f}% \n")

In [32]:
def print_train_time(start, end, device):
    total_time = end - start
    print(f"\nTotal training time: {total_time:.3f}s on {device}")
    return total_time

torch.manual_seed(42)
from timeit import default_timer as timer
from tqdm.auto import tqdm

start_time = timer()
epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch:{epoch+1}/{epochs}\n-----")
    train_step(model=model,
               data_loader=trainloader,
               optimizer=optimizer,
               loss_fn=loss_fn,
               scheduler=scheduler,
               accuracy_fn=accuracy_fn,
               device=device)
    test_step(model=model,
              data_loader=testloader,
              loss_fn=loss_fn,
              accuracy_fn=accuracy_fn,
              device=device)
    
end_time = timer()
total_time = print_train_time(start=start_time, end=end_time, device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:1/5
-----


train loss: 2.00428  | train acc: 0.46166% | lr: 0.00100
test loss: 2.59241 | test acc:0.35758% 

Epoch:2/5
-----
train loss: 1.84492  | train acc: 0.49612% | lr: 0.01000
test loss: 2.40049 | test acc:0.39341% 

Epoch:3/5
-----
train loss: 1.69905  | train acc: 0.53145% | lr: 0.00100
test loss: 2.32742 | test acc:0.40834% 

Epoch:4/5
-----
train loss: 1.56387  | train acc: 0.56356% | lr: 0.01000
test loss: 2.67159 | test acc:0.38167% 

Epoch:5/5
-----
train loss: 1.43925  | train acc: 0.59353% | lr: 0.00100
test loss: 2.38776 | test acc:0.41391% 


Total training time: 122.991s on cuda
